In [ ]:
!pip install transformers
!pip install tensorboardX
# !pip install wikipedia
# !pip install swifter
!pip install gdown

In [67]:
import torch
import tensorflow as tf
import pandas as pd
# import wikipedia 
# import swifter
import numpy as np

## GPU Detection

In [68]:
# If there is a GPU available
if torch.cuda.is_available():    
    # Tell PyTorch to use GPU
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('GPU in use:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
GPU in use: Tesla T4


## Import, Parse, and Store Data

In [1]:
#Creating PyDrive instance to load in data from PeTaL shared drive, follow the steps to authenticate
!pip install -U -q PyDrive 
  
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 
  
# Authenticate and create the PyDrive client. 
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [ ]:
import gdown

url = 'https://drive.google.com/file/d/1eOLNOl6ZMz4UxQ7qbSI-bJSSNkmNZjr9/view?usp=sharing'
output = 'SINGLELABEL_Colleen_and_Alex_training_data_4.19.csv'
gdown.download(url, output, quiet=False)

md5 = 'fa837a88f0c40c513d975104edf3da17'
gdown.cached_download(url, output, md5=md5, postprocess=gdown.extractall)

In [ ]:
# labeled_df_link = 'https://drive.google.com/file/d/1MJDIPe1C0dFHIPWu0w18IEJhVk4Xbk2x/view?usp=sharing'
# 'https://drive.google.com/file/d/1OZnAk64SPXfnaEzQFfhDJd6AX3dntIy9/view?usp=sharing'
# labeled_id = labeled_df_link.split("/")[-2]
# labeled_downloaded = drive.CreateFile({'id':labeled_id})  
# labeled_downloaded.GetContentFile('single_label.csv') 
# 'Biological-Strategies-Export-2020-October-01-1849 (1).csv'
# labeled_df = pd.read_csv('single_label.csv')

In [7]:
# new_labeled_df_link = 'https://drive.google.com/file/d/1tKxbJeMlJU_Dh62Xqgqdi7ua-AQRFEM9/view'
# new_labeled_id = new_labeled_df_link.split("/")[-2]
# new_labeled_downloaded = drive.CreateFile({'id':new_labeled_id})  
# new_labeled_downloaded.GetContentFile('labeled_abstracts_for_ML.csv') 
# new_labeled_df = pd.read_csv('labeled_abstracts_for_ML.csv')

In [70]:
sixhundreds_labeled_df_link = 'https://drive.google.com/file/d/1eOLNOl6ZMz4UxQ7qbSI-bJSSNkmNZjr9/view?usp=sharing'
sixhundreds_labeled_id = sixhundreds_labeled_df_link.split("/")[-2]
sixhundreds_labeled_downloaded = drive.CreateFile({'id':sixhundreds_labeled_id})  
sixhundreds_labeled_downloaded.GetContentFile('SINGLELABEL_Colleen_and_Alex_training_data_4.19.csv') 
sixhundreds_labeled_df = pd.read_csv('SINGLELABEL_Colleen_and_Alex_training_data_4.19.csv')
#sixhundreds_labeled_df.tail(50)

In [9]:
# labeled_df = labeled_df[['id', 'Title', 'Living Systems', 'Sources_source_link', 'Functions', 'Wikipedia', 'pdf_links', 'single_label']]
# labeled_df = labeled_df[labeled_df['Functions'].notnull( )]
# labeled_df = labeled_df[labeled_df['Sources_source_link'].notnull()]

In [10]:
# new_labeled_df = new_labeled_df[['title', 'abstract', 'labels', 'doi']]
# new_labeled_df = new_labeled_df[labeled_df['labels'].notnull( )]
# new_labeled_df = new_labeled_df[labeled_df['abstracts'].notnull()]

In [71]:
sixhundreds_labeled_df = sixhundreds_labeled_df[['title', 'abstract', 'labels', 'doi', 'url', 'single_labels', 'labels_string']]
sixhundreds_labeled_df = sixhundreds_labeled_df[sixhundreds_labeled_df['single_labels'].notnull( )]

In [ ]:
# labeled_df

In [ ]:
# new_labeled_df

In [ ]:
sixhundreds_labeled_df

In [ ]:
# All labels in dataset sorted by frequency
# labeled_df['single_label'] = labeled_df.apply(lambda x: x['single_label'].split('|')[0], axis=1)
# labeled_df['single_label'].value_counts().index

In [ ]:
# All labels in NEW 300s dataset sorted by frequency
# new_labeled_df['labels'] = new_labeled_df.apply(lambda x: x['labels'].split('|')[0], axis=1)
# new_labeled_df['labels'].value_counts().index

In [8]:
# All labels in NEW 600s ARTICLES dataset sorted by frequency
#sixhundreds_labeled_df['single_labels'] = sixhundreds_labeled_df.apply(lambda x: x['single_labels'].split('|')[0], axis=1)
sixhundreds_labeled_df['single_labels'].value_counts().index

Index([''Protect from harm'', ''Process resources'',
       ''Sense send or process information'',
       ''Maintain structural integrity'', ''Move'', ''Attach'',
       ''Maintain ecological community'',
       ''Chemically modify or Change energy state'', ''Change size or color'',
       ''Physically assemble/disassemble''],
      dtype='object')

In [73]:
# Splitting labels for training and testing, setting up data

import re
import string

labels = []
docs = []
labels_test = []
docs_test = []
labels_dict = ["'Protect from harm'", "'Process resources'", "'Sense send or process information'", "'Maintain structural integrity'", "'Move'", "'Attach'", "'Maintain ecological community'", "'Chemically modify or Change energy state'", "'Change size or color'", "'Physically assemble/disassemble'"]

single_labels = sixhundreds_labeled_df["single_labels"].tolist()
abstract = sixhundreds_labeled_df["abstract"].tolist()
title = sixhundreds_labeled_df["title"].tolist()

for i in range(len(title)):
  if i < len(title) - 40:
    docs.append(abstract[i])
    labels.append(labels_dict.index(single_labels[i]))
  else:
    docs_test.append(abstract[i])
    labels_test.append(labels_dict.index(single_labels[i]))


print ("Number of training labels: {:}".format(len(labels)))
print ("Number of training docs: {:}".format(len(docs)))
print ("Number of test labels: {:}".format(len(labels_test)))
print ("Number of test docs: {:}".format(len(docs_test)))
print(labels)
print(docs)
print(labels_test)
print(docs_test)

Number of training labels: 483
Number of training docs: 483
Number of test labels: 40
Number of test docs: 40
[3, 2, 1, 3, 1, 6, 8, 0, 8, 5, 0, 7, 0, 0, 1, 0, 3, 0, 0, 1, 2, 1, 5, 2, 6, 0, 0, 5, 9, 5, 3, 0, 1, 0, 0, 0, 7, 1, 0, 2, 2, 2, 5, 2, 7, 0, 2, 3, 0, 0, 1, 3, 1, 1, 0, 3, 4, 0, 5, 0, 3, 1, 0, 2, 5, 5, 9, 6, 4, 8, 4, 3, 3, 6, 2, 8, 4, 0, 6, 7, 3, 0, 2, 7, 2, 5, 1, 7, 0, 3, 6, 5, 9, 2, 6, 1, 0, 8, 5, 3, 1, 8, 1, 6, 1, 4, 8, 6, 7, 2, 3, 0, 6, 1, 4, 5, 0, 2, 9, 2, 5, 1, 3, 6, 8, 4, 6, 3, 5, 3, 1, 5, 6, 5, 2, 6, 0, 8, 1, 0, 2, 0, 0, 4, 8, 1, 0, 1, 1, 7, 2, 9, 4, 3, 9, 2, 6, 2, 4, 0, 1, 6, 0, 1, 0, 2, 6, 0, 5, 0, 2, 0, 0, 2, 2, 7, 4, 9, 4, 1, 1, 2, 4, 0, 1, 0, 0, 0, 3, 8, 3, 0, 0, 4, 0, 5, 3, 5, 5, 0, 7, 6, 3, 0, 0, 2, 1, 5, 0, 7, 1, 0, 7, 3, 5, 5, 1, 0, 7, 4, 7, 2, 0, 4, 3, 5, 5, 1, 5, 1, 9, 0, 3, 6, 9, 4, 2, 6, 1, 4, 4, 5, 1, 3, 7, 8, 9, 6, 0, 1, 0, 1, 7, 5, 4, 4, 2, 1, 6, 4, 0, 0, 3, 2, 2, 5, 0, 4, 5, 2, 5, 2, 2, 6, 1, 3, 0, 5, 7, 4, 5, 7, 0, 4, 6, 1, 0, 6, 3, 4, 0, 4, 3, 4, 6, 9, 4

In [ ]:
# Splitting data code for previous dataset
# import re
# import string

# labels = []
# docs = []
# labels_test = []
# docs_test = []
# labels_dict = ['Protect from harm', 'Move', 'Process information', 'Attach', 'Maintain structural integrity']

# title = new_labeled_df["title"].tolist()
# abstract = new_labeled_df["abstract"].tolist()
# labels = new_labeled_df["labels"].tolist()
# doi = new_labeled_df["doi"].tolist()
# for i in range(len(title)):
#   if i < len(title) - 73: #310, 36
#     if labels[i] == labels_dict[0]:
#       docs.append(abstract[i])
#       labels.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[1]:
#       docs.append(abstract[i])
#       labels.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[2]:
#       docs.append(abstract[i])
#       labels.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[3]:
#       docs.append(abstract[i])
#       labels.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[4]:
#       docs.append(abstract[i])
#       labels.append(labels_dict.index(labels[i]))
#   else:
#     if labels[i] == labels_dict[0]:
#       docs_test.append(abstract[i])
#       labels_test.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[1]:
#       docs_test.append(abstract[i])
#       labels_test.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[2]:
#       docs_test.append(abstract[i])
#       labels_test.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[3]:
#       docs_test.append(abstract[i])
#       labels_test.append(labels_dict.index(labels[i]))
#     if labels[i] == labels_dict[4]:
#       docs_test.append(abstract[i])
#       labels_test.append(labels_dict.index(labels[i]))

# print ("Number of training labels: {:}".format(len(labels)))
# print ("Number of training docs: {:}".format(len(docs)))
# print ("Number of test labels: {:}".format(len(labels_test)))
# print ("Number of test docs: {:}".format(len(docs_test)))
# print(labels)
# print(docs)
# print(labels_test)
# print(docs_test)

## Helper Functions

In [74]:
# Calculate accuracy of predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [75]:
# Format elapsed times as hh:mm:ss
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

## SciBERT Tokenizer

In [76]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
#model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

In [77]:
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModel

print('Loading SciBERT tokenizer')
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

Loading SciBERT tokenizer


In [16]:
## FOR REFERENCE: original BERT tokenizer
#from transformers import BertTokenizer, BertModel, BertConfig

#print('Loading BERT tokenizer')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer


In [ ]:
# tokenization example
# original abstract
print(' Original: ', docs[5])

# abstract split into tokens
print('Tokenized: ', tokenizer.tokenize(docs[5]))

# abstract as mapped to ids
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(docs[5])))

In [79]:
max_len = 0

for d in docs:
    # tokenize text and add `[CLS]` and `[SEP]` tokens
    input_ids = tokenizer.encode(d, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print('Max length: ', max_len)

Max length:  838


In [ ]:
# Finishing tokenizing all docs and map tokens to thier word IDs
input_ids = []
attention_masks = []

for d in docs:

    encoded_dict = tokenizer.encode_plus(
                        d,                      
                        truncation=True,
                        add_special_tokens = True, 
                        max_length = 256,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


labels = torch.tensor(labels) #, dtype=torch.long

print('Original: ', docs[5])
print('Token IDs:', input_ids[5])
print('Reverse:', tokenizer.convert_ids_to_tokens(input_ids[5]))

In [ ]:
# Split up training & testing/validation

from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

# 80:20 split

# Number of docs to include per set
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training docs'.format(train_size))
print('{:>5,} validation docs'.format(val_size))

In [82]:
# Iterator using torch DataLoader class so that entire dataset doesn't need to be stored in memory

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# batch size can be 16 or 32
batch_size = 32

# Sample in random order when training
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

## Training the Classification Model w/ Sequence Classification
  (fine-tune BERT)

  [HuggingFace documentation](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html)

In [83]:
from transformers import AutoModel, AutoModelForSequenceClassification, BertForSequenceClassification, BertConfig
  
model = AutoModelForSequenceClassification.from_pretrained(
    "allenai/scibert_scivocab_uncased", 
    num_labels = 10,  
    output_attentions = False, 
    output_hidden_states = False, 
)

model.save_pretrained('results/tokenizer/')
model.resize_token_embeddings(len(tokenizer))

# this needs to be run on GPU
model.cuda()

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [84]:
params = list(model.named_parameters())

print('model has {:} different named parameters\n'.format(len(params)))

print('Embedding Layer: \n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\nFirst Transformer:\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\nOutput Layer:\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

model has 201 different named parameters

Embedding Layer: 

bert.embeddings.word_embeddings.weight                  (31090, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

First Transformer:

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (768,)
bert.encoder.layer.0.

In [ ]:
# FOR REFERENCE: older BERT model
# from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification -> BERT model w/ added classification layer 
# model = BertForSequenceClassification.from_pretrained(
#    "bert-base-uncased", # 12-layer model, uncased vocab
#    num_labels = 10, # Number of labels 
#    output_attentions = False, 
#    output_hidden_states = False, 
# )
# model.save_pretrained('results/tokenizer/')
# model.resize_token_embeddings(len(tokenizer))

# this needs to be run on GPU
# model.cuda()

## Optimizer for our hypermarameters / Learning Rate Scheduler
AdamW

Possible hyperparamters: 
* batch size: 16, 32
* learning rate: 5e-5, 3e-5, 2e-5
* number of epochs: 2, 3, 4

In [85]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # 2e-5, 
                  eps = 1e-8 
                )

In [86]:
from transformers import get_linear_schedule_with_warmup

# Training epochs should be betw 2- 4 (reduce if overfitting)
epochs = 3 # 4

total_steps = len(train_dataloader) * epochs

# LR scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

## Training Loop

In [87]:
import random
# based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
loss_values = []

for epoch_i in range(0, epochs):

    print("")
    print('Epoch {:} / {:} '.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0

    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # `batch` pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        

        loss = outputs[0]

        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)            
    
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch time: {:}".format(format_time(time.time() - t0)))
        

    print("")
    print(" Validation ->")
    t0 = time.time()

    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():        

            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation time: {:}".format(format_time(time.time() - t0)))
    training_time = format_time(time.time() - t0)
    validation_time = format_time(time.time() - t0)

    training_stats = []
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Accur.': eval_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("training complete")
print("")


Epoch 1 / 3 
Training...

  Average training loss: 2.19
  Training epoch time: 0:00:18

 Validation ->
  Accuracy: 0.30
  Validation time: 0:00:01

Epoch 2 / 3 
Training...

  Average training loss: 1.81
  Training epoch time: 0:00:18

 Validation ->
  Accuracy: 0.45
  Validation time: 0:00:01

Epoch 3 / 3 
Training...

  Average training loss: 1.45
  Training epoch time: 0:00:19

 Validation ->
  Accuracy: 0.56
  Validation time: 0:00:01

training complete



In [ ]:
import pandas as pd
pd.set_option('precision', 2)

df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')

#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

df_stats

In [ ]:
# loss graph
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

In [61]:
df = df[['docs_test', 'labels_test']]

print('Number of test docs: {:,}\n'.format(df.shape[0]))

docs = df.abstract.values
labels = df.single_labels.values

Number of test docs: 523



In [ ]:
input_ids_test = []
attention_masks_test = []
actual_labels_test=[]

for i in range(40):

    encoded_dict = tokenizer.encode_plus(
                        docs_test[i],                      
                        add_special_tokens = True, 
                        max_length = 256,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
    
   
    input_ids_test.append(encoded_dict['input_ids'])
    

    attention_masks_test.append(encoded_dict['attention_mask'])
    actual_labels_test.append(labels_test[i])

# lists -> tensors
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
actual_labels_test = torch.tensor(actual_labels_test, dtype=torch.long)

batch_size = 32  

# build DataLoader
prediction_data = TensorDataset(input_ids_test, attention_masks_test, actual_labels_test)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

## Testing Classification

In [ ]:
print('Label predictions for {:,} test publications...'.format(len(input_ids_test)))
model.eval()

predictions, actual_labels = [], []

for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  # save memory and accelerate predictions w/o storing gradients
  with torch.no_grad():
      # forward pass and logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]

  # move logits and labels -> CPU
  logits = logits.detach().cpu().numpy()
  labels_ids_test = b_labels.to('cpu').numpy()
  
  predictions.append(logits)
  actual_labels.append(labels_ids_test)

classification_correct = 0

for i in range(len(predictions)):
  for j in range(len(predictions[i])):
    prediction = np.argmax(predictions[i][j])
    print ('Prediction: ' , prediction , ', actual: ', actual_labels[i][j])
    if prediction == actual_labels[i][j]:
      classification_correct = classification_correct + 1

print ('Classified correctly: ',  classification_correct)

print ('Model accuracy from testing: {0:.2f}'.format(classification_correct / len(input_ids_test)))

In [ ]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

print('matthews corr. coef. functions...')

for i in range(len(true_labels)):
  
  # converting batch predictions
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

In [ ]:
# barplot showing the MCC score for each batch of test samples.
ax = sns.barplot(x=list(range(len(matthews_set))), y=matthews_set, ci=None)

plt.title('MCC Score per Batch')
plt.ylabel('MCC Score (-1 to +1)')
plt.xlabel('Batch #')

plt.show()

In [ ]:
!pip install lit-nlp

In [ ]:
# save model files
import os

output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("save to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  
model_to_save.save_pretrained(output_dir)

tokenizer.save_pretrained(output_dir)

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls -l --block-size=K ./model_save/

total 428008K
-rw-r--r-- 1 root root      1K Feb 23 18:52 config.json
-rw-r--r-- 1 root root 427768K Feb 23 18:52 pytorch_model.bin
-rw-r--r-- 1 root root      1K Feb 23 18:52 special_tokens_map.json
-rw-r--r-- 1 root root      1K Feb 23 18:52 tokenizer_config.json
-rw-r--r-- 1 root root    227K Feb 23 18:52 vocab.txt


In [ ]:
!ls -l --block-size=M ./model_save/pytorch_model.bin

-rw-r--r-- 1 root root 418M Feb 23 18:52 ./model_save/pytorch_model.bin


In [ ]:
!cp -r ./model_save/ "./drive/My Drive/Model"

In [ ]:
# load pretrained model
model = model.from_pretrained(output_dir)
tokenizer = tokenizer.from_pretrained(output_dir)

# copy to gpu
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
PATH = "state_dict_model.pt"

# Save
torch.save(model.state_dict(), PATH)

#input_ids = []
#for d in docs:
#
#    encoded_dict = tokenizer.encode_plus(
#                        d,                      
#                        truncation=True,
#                        add_special_tokens = True, 
#                        max_length = 256,           
#                        pad_to_max_length = True,
#                        return_attention_mask = True,   
#                        return_tensors = 'pt',     
#                   )
#    
#    input_ids.append(encoded_dict['input_ids'])
#
#input_ids = torch.cat(input_ids, dim=0)

# Load
model = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
#(input_ids, attention_masks)
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
PATH = "model.pt"

# Save
torch.save(model, PATH)

# Load
model = torch.load(PATH)
model.eval()

In [ ]:
torch.save(model.state_dict(), 'model.pt')

In [ ]:
import sagemaker
from sagemaker.pytorch import model.PyTorchModel

pytorch_model = PyTorchModel ( model_data=model_data,
                    role=role,
                     framework_version='1.0.0',
                     py_version="py3"
)

In [ ]:
estimator = PyTorch(entry_point=' ',
                     role=role,
                     framework_version='1.4.0',
                     train_instance_count=1,
                     train_instance_type=' ',
                     source_dir=' s',
                     git_config=git_config,
                     # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                     #                            bptt, dropout, tied, seed, log_interval
                     hyperparameters={
                         'epochs': 1,
                         'tied': True
                     })


In [ ]:
import tarfile
import os.path

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))